Scrape an online E-Commerce Site for Data. 
1. Extract product data from Amazon - be it any product and put these 
details in the MySQL database. One can use pipeline. Like 1 pipeline to 
process the scraped data and other to put data in the database and since 
Amazon has some restrictions on scraping of data, ask them to work on small 
set of requests otherwise proxies and all would have to be used. 
2. Scrape the details like color, dimensions, material etc. Or customer ratings 
by features.

In [ ]:
pip install kora -q

     |████████████████████████████████| 57 kB 2.8 MB/s 
     |████████████████████████████████| 59 kB 6.2 MB/s 


In [ ]:
import csv
from bs4 import BeautifulSoup

In [ ]:
from kora.selenium import wd
wd.get('https://www.amazon.in/')

In [ ]:
wd.page_source

'<html lang="en-in" class=" a-js a-audio a-video a-canvas a-svg a-drag-drop a-geolocation a-history a-webworker a-autofocus a-input-placeholder a-textarea-placeholder a-local-storage a-gradients a-transform3d a-touch-scrolling a-text-shadow a-text-stroke a-box-shadow a-border-radius a-border-image a-opacity a-transform a-transition null" data-19ax5a9jf="dingo" data-aui-build-date="3.22.1-2022-05-25" data-useragent="Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/537.36 (KHTML, like Gecko) HeadlessChrome/101.0.4951.64 Safari/537.36" data-platform="Linux x86_64"><!-- sp:feature:head-start --><head><script async="" src="https://c.amazon-adsystem.com/bao-csm/forensics/a9-tq-forensics-incremental.min.js" crossorigin="anonymous"></script><script async="" src="https://images-eu.ssl-images-amazon.com/images/I/31YXrY93hfL.js" crossorigin="anonymous"></script><script>var aPageStart = (new Date()).getTime();</script><meta charset="utf-8">\n<!-- sp:end-feature:head-start -->\n\n<script type="text/java

In [ ]:
from selenium import webdriver
options = webdriver.ChromeOptions()
options.add_argument('-headless')
options.add_argument('-no-sandbox')
options.add_argument('-disable-dev-shm-usage')

In [ ]:
wd=webdriver.Chrome('chromedriver',options=options)
wd.get("https://www.amazon.in/")
print(wd.page_source)

<html lang="en-in" class=" a-js a-audio a-video a-canvas a-svg a-drag-drop a-geolocation a-history a-webworker a-autofocus a-input-placeholder a-textarea-placeholder a-local-storage a-gradients a-transform3d a-touch-scrolling a-text-shadow a-text-stroke a-box-shadow a-border-radius a-border-image a-opacity a-transform a-transition null" data-19ax5a9jf="dingo" data-aui-build-date="3.22.1-2022-05-25" data-useragent="Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/537.36 (KHTML, like Gecko) HeadlessChrome/101.0.4951.64 Safari/537.36" data-platform="Linux x86_64"><!-- sp:feature:head-start --><head><script async="" src="https://c.amazon-adsystem.com/bao-csm/forensics/a9-tq-forensics.min.js" crossorigin="anonymous"></script><script async="" src="https://images-eu.ssl-images-amazon.com/images/I/31YXrY93hfL.js" crossorigin="anonymous"></script><script>var aPageStart = (new Date()).getTime();</script><meta charset="utf-8">
<!-- sp:end-feature:head-start -->

<script type="text/javascript">var ue_t

In [ ]:
def get_url(search_term):
  template="https://www.amazon.in/s?k={}&rh=n%3A1389401031&ref=nb_sb_noss/"
  searc_term = search_term.replace(' ', '+')
  return template.format(search_term)

In [ ]:
url = get_url('laptop')
print(url)

https://www.amazon.in/s?k=laptop&rh=n%3A1389401031&ref=nb_sb_noss/


In [ ]:
wd.get(url)
soup = BeautifulSoup(wd.page_source, 'html.parser')

result=soup.find_all('div',{'data-component-type':'s-search-result'})
len(result)
print(result[0])

<div class="s-result-item s-asin sg-col-0-of-12 sg-col-16-of-20 sg-col s-widget-spacing-small sg-col-12-of-16" data-asin="B07GS3ZT32" data-component-id="11" data-component-type="s-search-result" data-index="7" data-uuid="8c90ac4b-7cb9-4033-8d59-2a704a009906"><div class="sg-col-inner"><div cel_widget_id="MAIN-SEARCH_RESULTS-7" class="s-widget-container s-spacing-small s-widget-container-height-small celwidget slot=MAIN template=SEARCH_RESULTS widgetId=search-results_1" data-csa-c-id="iaia9b-mhxze8-7cz4ks-kozv25"><div class="s-card-container s-overflow-hidden aok-relative s-include-content-margin s-latency-cf-section s-card-border"><div class="a-section"><div class="sg-row"><div class="sg-col sg-col-4-of-12 sg-col-4-of-16 sg-col-4-of-20 s-list-col-left"><div class="sg-col-inner"><div class="a-section a-spacing-none aok-relative s-image-overlay-grey s-list-status-badge-container"></div><div class="s-product-image-container aok-relative s-image-overlay-grey s-text-center s-padding-left-sma

In [ ]:
item=result[0]

In [ ]:
atag=item.h2.a

In [ ]:
atag.text

'De Bienes Kart 3 Pack Webcam Cover Privacy Sticker - Removable & Reusable Web Camera Cover for Android PC Laptops MacBooks iOS Smartphones iPad Tablet ect -T1 Black '

In [ ]:
price_parent=item.find('span', 'a-price')
price_parent.find('span','a-offscreen').text

'₹199'

In [ ]:
rating = item.i.text
print(rating)

3.8 out of 5 stars


In [ ]:
review_count = item.find('span', {'class':'a-size-base','dir':'auto'})

print(review_count)

None


In [ ]:
def extract_record(item1):
  atag = item1.h2.a
  description = atag.text.strip()
  url = "https://www.amazon.in/" + atag.get('href')

  price_parent = item1.find('span','a-price')
  #price_parent.find('span','a-offscreen').text

  rating = ""
  result = (description, price_parent, rating)
  return result

In [ ]:
url = get_url('mouse')
wd.get(url)
soup = BeautifulSoup(wd.page_source, 'html.parser')
records = []
results = soup.find_all('div',{'data-component-type':'s-search-result'})

In [ ]:
for item in results:
  records.append(extract_record(item))
records[0]
print("printing records")
for x in range(len(records)):
  print(records[x])

printing records
('SAFETYNET GSM USB PC Mouse Calling Device with GSM Listening 2-Way Audio Bug Surveillance Device(GSM Sim Card) Mouse .Clear Listening Sound', <span class="a-price" data-a-color="price" data-a-size="l"><span class="a-offscreen">₹3,440</span><span aria-hidden="true"><span class="a-price-symbol">₹</span><span class="a-price-whole">3,440</span></span></span>, '')
('Red Champion Wired Black USB Wired Plug & Play Ergonomic Mouse with Type C OTG & Micro OTG Wire', <span class="a-price" data-a-color="price" data-a-size="l"><span class="a-offscreen">₹349</span><span aria-hidden="true"><span class="a-price-symbol">₹</span><span class="a-price-whole">349</span></span></span>, '')
('BenQ Zowie CAMADE II Bungee for e-Sports Mice Cable Management Device Black/Red', <span class="a-price" data-a-color="price" data-a-size="l"><span class="a-offscreen">₹2,459</span><span aria-hidden="true"><span class="a-price-symbol">₹</span><span class="a-price-whole">2,459</span></span></span>, '')